In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("meteostat_kn.csv")
df.columns = ['date', 'hour', 'temp', 'dew_point', 'humidity', 'precipitation', 'snow', 'wind_direction', 'wind_speed', 'wind_gust', 'pressure', 'sunshine_minutes', 'code']

# Remove rows where the date is newer than the current date
current_date = datetime.now().date()
df['date'] = pd.to_datetime(df['date'])  # Ensure 'date' column is in datetime format
df = df[df['date'] < datetime(current_date.year, current_date.month, current_date.day)]
df.tail()

In [ ]:
df["temp_dew_point_diff"] = df["temp"] - df["dew_point"]

In [ ]:
features = ['hour', 'temp', 'dew_point', 'temp_dew_point_diff', 'humidity', 'pressure', 'wind_speed', 'wind_gust']
X = df[features]
y = df['code']

In [ ]:
# Combine X and y into a single DataFrame for easier handling of NaN values
data = pd.concat([X, y], axis=1)

# Drop rows where any feature or the target is NaN
data = data.dropna()

# Separate X and y again
X = data[features]

# Create binary target: 1 if code is 5 (fog) or 6 (freezing fog), else 0
y = data['code'].apply(lambda x: 1 if x == 5 or x == 6 else 0)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define parameter grid
param_grid = {
    'max_depth': [3, 5, 10],
    'n_estimators': [50, 100, 200, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.3, 0.5]
}

# Recalculate imbalance ratio
ratio = (y_train == 0).sum() / (y_train == 1).sum()

xgb_model = XGBClassifier(
    scale_pos_weight=ratio,
    eval_metric='logloss',
    early_stopping_rounds=10
)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    verbose=False,
    n_jobs=-1
)

# Run grid search
grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_val, y_val)], verbose=False)

In [ ]:
print("Best parameters:", grid_search.best_params_)
print("Best F1 score:", grid_search.best_score_)

best_model = grid_search.best_estimator_
results = best_model.evals_result()

epochs = range(len(results['validation_0']['logloss']))

plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, results['validation_0']['logloss'], label='Train Loss')
plt.plot(epochs, results['validation_1']['logloss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Log Loss')
plt.title('Log Loss over Epochs')
plt.legend()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

y_pred = best_model.predict(X_test)

cm = confusion_matrix(y_test, y_pred, labels=[1, 0])  # 1=foggy, 0=not foggy
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Foggy", "Not Foggy"])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Get indices of false positives (predicted 1, actual 0)
false_positives = X_test[(y_pred == 1) & (y_test == 0)].head(10)

# Get indices of false negatives (predicted 0, actual 1)
false_negatives = X_test[(y_pred == 0) & (y_test == 1)].head(10)

print("False Positives:")
print(false_positives)

print("\nFalse Negatives:")
print(false_negatives)

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

y_scores = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class (foggy)

precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
ap = average_precision_score(y_test, y_scores)

plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve (AP = {ap:.2f})')
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = roc_auc_score(y_test, y_scores)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
best_model.save_model("../backend/forecast/xgb_foggy.json")